# Building RAG Chatbot for Technical Documentation



##### Guilherme Vaz, Sebastião Santos Lessa
This project centers on utilizing the **Haystack framework**, a flexible and comprehensive open-source library in Python, renowned for its capabilities in constructing RAG pipelines and combining model retrieval with generation functionalities. Haystack’s modular architecture supports seamless integration with other technologies, such as those offered by Hugging Face, enabling the implementation of sophisticated search and generation processes. To enhance model interactions, the **Hugging Face inference client** was incorporated, improving API communication and supporting efficient interaction between Python applications and hosted models. For testing, we used the complete text of the **European Union Medical Device Regulation - Regulation (EU) 2017/745 (EU MDR)** to evaluate the system’s effectiveness and accuracy in handling complex, regulatory content.

The core LLM used is **meta-llama/Llama-3.2-1B-Instruct**, known for its instruction-following proficiency
and balanced performance, offering an optimal blend of response quality and computational efficiency.
Additionally, we used **Few-Shot Example Prompts** within system prompts to guide the model’s
responses. This technique involved providing multiple examples within the system prompts, allowing
the LLM to better understand the expected response format and improve contextual accuracy.Nuanced
prompt refinements were also added in the final user prompt to increase relevance and depth in responses.

**To run the notebook, it must be placed inside the haystack_components folder within the zip file provided. Please follow the instructions in the README to set up the databases correctly. Alternatively, to run the project without the notebook, you must clone the GitHub repository we’ve provided.**

In [1]:
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever
from haystack_integrations.document_stores.opensearch import OpenSearchDocumentStore
from haystack_integrations.components.retrievers.opensearch import OpenSearchBM25Retriever, OpenSearchEmbeddingRetriever
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter, TextCleaner


from documents_pipeline.classifiers import NamedEntityExtractor  #,IntentExtractor

from documents_pipeline.Splitter import LayoutPDFSplitter

from prompt_re_eng.llm import LLMPrompt
from documents_pipeline.save_stores import get_Osearch_store, get_qdrant_store, save_docs_to_Osearch, \
    save_docs_to_QDRANT
from prompt_re_eng.new_search import QdrantSearch, OpenSearch, JoinDocuments  #, SentenceTransformersRanker
from askLLM.GPT import ASK_LLM
import time

c:\Users\guiva\Documents\Aulas\Master\TAAC\RAG_TAAC\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **Processing Documents Pipeline**

The document processing pipeline is designed to optimize the way documents are ingested, segmented,
and stored for efficient retrieval and analysis. The pipeline operates as follows:
The main steps include:

    • Document Segmentation Using LLM Sherpa: Unlike the default document splitting methods
    used by frameworks like Haystack or LangChain, which may randomly split documents or rely
    on fixed-size chunking, this pipeline leverages the LLM Sherpa model. This model identifies and
    segments the text from one header to the next, preserving the document’s logical structure and
    ensuring that each section is contextually coherent.
    
    • NER Classification for Metadata Generation: Once the document is divided into sections,
    each segment undergoes Named Entity Recognition (NER) using a SpaCy model. This step extracts
    entities and classifies them to enrich the content with relevant metadata. These metadata tags
    enhance the searchability and contextual understanding of the document, enabling more refined
    and precise search results.

    • Keyword-Based Storage: The sections are stored in a database indexed by keywords, supporting
    more efficient retrieval based on thematic relevance and allowing for targeted search queries that
    yield high-value responses.

    • Segmenting Sections into Sentences: To enhance the granularity and effectiveness of the vector
    database, each section is split into individual sentences. Storing large sections as a single vector
    representation can dilute semantic specificity and reduce search precision. By breaking sections
    down into sentences, the pipeline ensures that each entry in the vector database maintains finegrained semantic resolution.

    • Storage in Vector Database: The processed sentences, enriched with metadata and keywords,
    are stored in the vector database. This enables semantic search capabilities, where the retrieval process leverages the embeddings of each sentence to find matches based on conceptual similarity rather
    than exact keyword matching, resulting in nuanced and contextually relevant query responses.

Contextual Note: Initially, an intention classification step was part of the pipeline, designed to categorize sections based on intent. However, after comparing results, it was determined that this step did
not add significant value. Therefore, it was removed to streamline the process and avoid unnecessary
complexity.

The final pipeline architecture is depicted in Figure 1, which illustrates the flow from document input
through segmentation, classification, and storage in both keyword and vector databases.


![Descrição da Imagem](../images/image1.png)


In [2]:

def document_processor_pipeline(doc_path, doc_id=1):
    """
    Processa e guarda um documento nas bases de dados.

    As linhas comentadas correspondem à forma default de fazer chunck do haystack.

    Parser Personalidado do LLMSherpa -> LLM que extrai NER e INTENT -> Guarda OpenSearch -> Divide em frases  -> Guarda QDrant
    
    """

    p = Pipeline()
    p.add_component("Splitter", LayoutPDFSplitter())
    p.add_component("NER", NamedEntityExtractor())

    p.add_component("Save_OS", save_docs_to_Osearch())
    p.add_component("Split_Sent", DocumentSplitter(split_by="sentence", split_length=3, split_overlap=0))
    p.add_component("Save_QD", save_docs_to_QDRANT())


    p.connect("Splitter", "NER")
    p.connect("NER", "Save_OS")
    p.connect("Save_OS", "Split_Sent")
    p.connect("Split_Sent", "Save_QD")

    res = p.run({"Splitter": {"sources": [doc_path], "doc_id": doc_id}})

    return res

### **Prompt Re-engineering**

The pipeline modifies the user prompt, adapting it into specific prompts optimized for different types
of searches, including keyword-based and vector-based searches. After these prompts are created, the
pipeline performs the search and retrieves the most relevant sections to answer the query. The process
follows these steps:

1. **User Prompt Transformation**:
    When a user prompt is received, it is processed by the LLM, which then generates specific multi
    sub-prompts tailored for both vector and keyword-based searches. This bifurcated approach ensures
    that the prompts aligns with the requirements of each search method and maximizes the retrieval
    capabilities of the system.
    
    **1.1 Prompts for Vector Database Search**: The generated prompts is sent to the vector database,
    where a semantic search is conducted. This type of search evaluates the conceptual similarity between
    the prompts and stored data, providing results that are contextually relevant.

    
    **1.2 Prompts for Keyword Database Search**: In parallel, the LLM also generates prompts suited
    for keyword-based searching. This search method looks for exact or thematic keyword matches in
    the keyword database, providing high-precision results for direct term correlations.
    
2. **Dual Search Execution**: Using these specialized prompts, the pipeline conducts multiple searches
in both the keyword-based and vector-based databases. The keyword searches focus on finding
documents through direct term matches, while vector searches retrieve contextually aligned results
by assessing semantic similarities.

3. **Result Concatenation**: After completing both searches, the pipeline concatenates the results from
the keyword and vector searches. This combined output leverages the precision of keyword matching
and the depth of semantic understanding from vector search, providing a comprehensive set of
results. The concatenation process is enhanced using ’JoinDocuments(join mode=”distribution
based rank fusion”)’. This method merges the outputs by evaluating the distribution of document
ranks from both searches, ensuring that the final result set reflects a balanced contribution from
each method. By considering the rank positions and their distribution, this approach maximizes
the relevance and diversity of the final response, yielding richer and more contextually accurate
outputs.

The overall architecture of this querying process is shown in Figure 2.

![Figure 2: Prompt Re-engineering](../images/image2.png)


In [3]:
def prompt_engineering_pipeline(prompt):
    """  
    Cria um dicionário com todas as informações relevantes para a pesquisa.  
    user_prompt -> LLM para otimizar a prompt para cada base de dados -> Pesquisa QDrant -> Pesquisa OpenSearch -> Combinação dos resultados  
    Formato final do dicionário prompt_mod:  
    {'original_prompt': '...', 'keyword_prompt': '...', 'vector_prompt': '...', 'vector_search_results': ... , 'vector_search_score': ... , 'keyword_search_result': ... , 'keyword_search_score': ...}  
    Posteriormente este dicionário vai ser dado a um LLM para ele conseguir responder à pergunta do user com este contexto extra.  
    """
    join_documents = JoinDocuments(join_mode="distribution_based_rank_fusion")

    p = Pipeline()
    p.add_component("LLM", LLMPrompt())
    p.add_component("VS", QdrantSearch())
    p.add_component("KS", OpenSearch())
    p.add_component("JoinDocuments", join_documents)

    p.connect("LLM", "VS")
    p.connect("LLM", "KS")
    p.connect("VS", "JoinDocuments")
    p.connect("KS", "JoinDocuments")
    res = p.run({"user_prompt": prompt})
    return res


### Querying the LLM

This stage of the pipeline involves the simplest yet critical task: submitting the processed user prompt
and the retrieved document sections to the LLM to generate a comprehensive and contextually relevant
response.

After the user prompt has been refined and the relevant document sections have been obtained through
keyword and vector-based searches, the system forwards these elements to the LLM. The LLM, equipped
with advanced language understanding and generation capabilities, synthesizes an output that integrates
the user’s query with the contextual information provided by the retrieved sections. This ensures that
the response is not only accurate but also context-aware, leveraging the depth of information found in
the database. To maintain efficiency and prevent overwhelming the model with irrelevant information,
we provide context up to a limit of 1000 tokens.

The key to this stage is the interaction between the LLM and the curated input data. By aligning the
user’s intent with supporting sections, the LLM can produce responses that are richer and more precise,
demonstrating an effective combination of retrieval and generation.

This approach highlights the simplicity of the final query phase while showcasing the effectiveness of
the prior steps. The resulting output reflects the LLM’s ability to draw from pre-retrieved, contextually
relevant data to enhance response accuracy and informativeness.

![Figure 2: Prompt Re-engineering](../images/image3.png)

In [4]:
def ask_LLM_with_context(prompt, context):
    p = Pipeline()
    p.add_component("LLM", ASK_LLM())
    res = p.run({"prompt": prompt, "context": context})
    return res

To demonstrate the practical utility of these components, we developed a full-stack web application
using Flask for server-side operations, paired with HTML and CSS to create a user-friendly browser
interface. This structure enabled real-time user interaction with the RAG system, highlighting the
project’s potential in practical deployment scenarios and showcasing the effective combination of modern
frameworks for local, accessible solutions

In [ ]:
from flask import Flask, request, render_template, jsonify, stream_with_context, Response
import os
from werkzeug.utils import secure_filename

def answer_question(prompt):
    return ask_LLM_with_context(prompt,prompt_engineering_pipeline(prompt))

app = Flask(__name__)
messages = []


@app.route('/', methods=['GET'])
def index():
    messages = []
    return render_template('index.html', messages=messages)


@app.route('/send_message', methods=['POST'])
def send_message():
    # Armazena a mensagem do usuário e responde imediatamente
    message = request.json.get('message', '')
    if message:
        messages.append(('sender', message))
        return jsonify({'status': 'received'})
    return jsonify({'status': 'error'}), 400


@app.route('/stream_response', methods=['POST'])
def stream_response():
    message = request.json.get('message', '')
    if not message:
        return jsonify({'error': 'No message provided'}), 400
    answer = answer_question(message)['LLM']['response']
    messages.append(('receiver', answer))
    return answer


# Configuração para armazenamento temporário de ficheiros
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)


@app.route('/upload_file', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'status': 'error', 'message': 'No file provided'}), 400
    file = request.files['file']
    print(file.filename)
    # Verificar se o ficheiro está presente e seguro para salvar
    if file.filename == '':
        return jsonify({'status': 'error', 'message': 'Empty file name'}), 400
    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(filepath)

    # Executar a pipeline nos documentos
    try:
        print(filepath)
        result = document_processor_pipeline(filepath)

        return jsonify({'status': 'success', 'result': result})
    except Exception as e:
        print(e)
        return jsonify({'status': 'error', 'message': str(e)}), 500


if __name__ == '__main__':
    app.run(debug=False, threaded=False)
